In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import keras
import os

#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Installing EfficientNet

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

## Importing EfficientNet

In [ ]:
from efficientnet.keras import EfficientNetB0
from efficientnet.keras import center_crop_and_resize, preprocess_input

## Loading Pretrained Model

In [ ]:
width = 150
height = 150
input_shape = (height, width, 3)


base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
train_dir = '../input/herbarium-2021-fgvc8/train/images'
test_dir = '../input/herbarium-2021-fgvc8/test/images'
batch_size = 512

## Using image data generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = True,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle = False,
        subset='validation')

print(train_generator.class_indices)

In [ ]:
import os, os.path
epochs = 1
NUM_TRAIN = sum([len(files) for r, d, files in os.walk(train_dir)])
dropout_rate = 0.2

In [ ]:
num_classes = len(os.listdir(train_dir))

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalMaxPooling2D(name="gmp"))
if dropout_rate > 0:
    model.add(layers.Dropout(dropout_rate, name="dropout"))
model.add(layers.Dense(num_classes, activation='softmax', name="out"))

In [ ]:
model.summary()

base_model.trainable = False

## Train model

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.01),
              metrics=['acc'])


history = model.fit(
      train_generator,
      steps_per_epoch= (NUM_TRAIN*0.8) //batch_size,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps= (NUM_TRAIN*0.2) //batch_size,
      verbose=1,
      use_multiprocessing=True,
      workers=4)

In [ ]:
model.save('effnetB0.h5')